In [1]:
import logging
logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s', level=logging.INFO)

In [2]:
def get_cuttext_mysql(sql_database, sql_table, cols_name, year):
    import mysql.connector
    sql_db = mysql.connector.connect(
        host = '127.0.0.1',
        user = 'root',
        password = "",
        database = sql_database
    )
    cursor = sql_db.cursor()
    
    # to check whether TABLE EXISTS OR NOT
    cursor.execute("SHOW TABLES LIKE '{}'".format(sql_table))
    temp_result = cursor.fetchone()
    if (temp_result):
        logging.info("TABLE {} exists in DATABASE {}".format(sql_table, sql_database))
    else:
        logging.info("TABLE {} exists in DATABASE {}".format(sql_table, sql_database))
        
    #select_sql = "SELECT {}, GROUP_CONCAT({} SEPARATOR '') FROM {} WHERE {}<'2021' GROUP BY {}".format(cols_name[0], cols_name[1], sql_table, cols_name[0], cols_name[0])
    #select_sql = "SELECT project_name, {} FROM {} WHERE {}='{}'".format(cols_name[1], sql_table, cols_name[0], year)
    select_sql = "SELECT project_id, {} FROM {} WHERE {}='{}'".format(cols_name[1], sql_table, cols_name[0], year)
    print(select_sql)
    cursor.execute(select_sql)
    result = cursor.fetchall()
    return result

In [3]:
# load cut word string into list
def word_to_list(cut_article):
    cut_word_list = []
    words = ""
    for c in cut_article:
        if c.isspace():
            cut_word_list.append(words)
            words = ""
        else:
            words += c
    return cut_word_list

In [4]:
# clean the stopword from flingv_description_cut.txt
def load_stopwords(filename):
    f = open(filename, "r", encoding='UTF-8')
    lines = f.readlines()
    for idx, val in enumerate(lines):
        lines[idx] = lines[idx].strip('\n')
    return lines

In [ ]:
# main block
# normal tf-idf, loop for each year
for y in range(2012, 2021):
    sql_database = "crowdfunding_db"
    #sql_table = "v3_latest_zeczec_basic_project_information"
    sql_table = "v6_latest_flyingv_basic_project_information"
    column_name = ["YEAR(start_date)", "description_text_cut"]
    cut_year = get_cuttext_mysql(sql_database, sql_table, column_name, y)
    logging.info("GET cut_text! year: {} length: {}".format(y, len(cut_year)))
    
    # cut the word & store as list & eliminate stopword
    cut_corpus = []
    stopword = load_stopwords('tc_stopwords.txt')
    for cy in cut_year:
        cut_article = cy[1].strip()
        cut_article = cut_article.replace('\n', '')
        cut_article_list = word_to_list(cut_article)

        no_stopword_list = []
        for w in cut_article_list:
            if (len(w)>=2) and (w not in stopword):
                no_stopword_list.append(w)
        cut_word_len = len(no_stopword_list)
        temp_list = [cy[0], cut_word_len, no_stopword_list]
        cut_corpus.append(temp_list)
    logging.info("cut_corpus DONE! year: {} length: {}".format(y, len(cut_year)))
    
    # tf-1
    # count occurence of each words in each project description
    words_count = []
    for p in cut_corpus:
        count = {}
        for word in p[2]:
            if word in count:
                count[word] += 1
            else:
                count[word] = 1
        words_count.append([p[0], p[1], count])
    words_frequency = []
    #tf-2
    for wc in words_count:
        all_count = sum(wc[2].values())
        fre = {}
        for word, count in wc[2].items():
            fre[word] = round(count/all_count, 4)
        words_frequency.append([wc[0], fre])
        
    # idf-1
    # count occurence of each words in each project description
    words_count = []
    for p in cut_corpus:
        count = {}
        for word in p[2]:
            if word in count:
                count[word] += 1
            else:
                count[word] = 1
        words_count.append(count)
    #print(len(words_count))

    # because wrods_count contains unique words for each description
    all_words = []
    for word in words_count:
        all_words.extend(list(word.keys()))
    #all_words[:10]
    # the occurence in desciption of each word
    occurences_of_word = {}
    for word in all_words:
        if word in occurences_of_word:
            occurences_of_word[word] += 1
        else:
            occurences_of_word[word] = 1
    #print(len(occurences_of_word.items()))
    # idf-2
    import math
    inverse_doc_fre = []
    print(len(words_count))
    for wc in words_count:
        inv_fre = {}
        for w in wc.keys():
            all_description = len(words_count)
            occurences = occurences_of_word[w]
            inv_fre[w] = math.log(round(all_description/occurences, 4)) # base=e
        inverse_doc_fre.append(inv_fre)
    
    # tf-idf
    all_tf_idf = []
    for i, p in enumerate(words_frequency):
        tf_idf = {}
        for word, freq in p[1].items():
            tf_idf[word] = freq*inverse_doc_fre[i][word]
        all_tf_idf.append([p[0], tf_idf])
    
    # keyword for the year: count the occurence of the word within top k keyword in each description
    # pick top k keyword from each description
    k = 100
    top_k_keyword = []
    for p in all_tf_idf:
        temp_word_list = sorted(p[1].items(), key=lambda item:item[1], reverse=True)
        #print(p[0], len(temp_word_list))
        if k>len(temp_word_list): # avoid not enough word in the description
            k=len(temp_word_list)
        if len(temp_word_list)!=0:
            for i in range(k):
                top_k_keyword.append(temp_word_list[i][0])
    print(len(top_k_keyword))
    # count the occurence of words in top_k_keyword
    occ_of_keyword = {}
    for word in top_k_keyword:
        if word in occ_of_keyword:
            occ_of_keyword[word] += 1
        else:
            occ_of_keyword[word] = 1
    #occ_of_keyword
    keyword_list = sorted(occ_of_keyword.items(), key=lambda item:item[1], reverse=True)
    print(keyword_list[:10])

    keyword_cols = ['word', 'occurence']
    keyword_df = pd.DataFrame(columns = keyword_cols)
    for w in keyword_list:
        temp_df = pd.DataFrame([[w[0], w[1]]], columns=keyword_cols)
        keyword_df = keyword_df.append([temp_df], ignore_index=True)
    #csv_filename = 'zeczec_{}_top_keyword_tfidf.csv'.format(y)
    csv_filename = 'flyingv_{}_top_keyword_tfidf.csv'.format(y)
    keyword_df.to_csv(csv_filename, encoding='utf-8-sig', index=False)
    print(keyword_df[:10])